In [9]:
from model.TKAN import Model

In [10]:
a = Model()

In [11]:
a.compile_execute_test()

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 83ms/step - loss: 0.2482
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.1447
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.1113
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - loss: 0.1065
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - loss: 0.1052
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - loss: 0.1079
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - loss: 0.0955
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - loss: 0.1014
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - loss: 0.0944
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - loss: 0.0978


In [13]:
import DataExtraction.meteocat as meteocat
meteocat.print_api_usage()

{'client': {'nom': '488 Mireia Miró Fatás UPC'}, 'plans': [{'nom': 'Referència Bàsic', 'periode': 'Mensual', 'maxConsultes': 2000, 'consultesRestants': 2000, 'consultesRealitzades': 0}, {'nom': 'Quota', 'periode': 'Mensual', 'maxConsultes': 300, 'consultesRestants': 300, 'consultesRealitzades': 0}, {'nom': 'XDDE_250', 'periode': 'Mensual', 'maxConsultes': 250, 'consultesRestants': 250, 'consultesRealitzades': 0}, {'nom': 'XEMA_750 OD', 'periode': 'Mensual', 'maxConsultes': 750, 'consultesRestants': 750, 'consultesRealitzades': 0}, {'nom': 'Predicció_100', 'periode': 'Mensual', 'maxConsultes': 100, 'consultesRestants': 100, 'consultesRealitzades': 0}]}
